In [6]:
import fastf1, datetime
import pandas as pd

fastf1.Cache.enable_cache('cache')
current_date = datetime.datetime.now()

In [7]:
strategy_df = pd.DataFrame(columns=['year', 'round', 'grand_prix', 'finish_position', 'number', 'driver', 'constructor', 'stint_compounds', 'stint_lengths', 'total_stops', 'total_laps'])

In [8]:
for year in range(2018, current_date.year + 1):
    schedule = fastf1.get_event_schedule(year)
    for _, event in schedule.iterrows():
        if event['EventFormat'] == 'testing':
            continue
        try:
            race = fastf1.get_session(year, event['EventName'], 'Race')
            race.load()

            race_date = race.date.to_pydatetime()
            if race_date > current_date:
                break

            laps = race.laps
            stint_data = (laps.groupby(['DriverNumber', 'Driver', 'Stint', 'Compound']).agg(stint_length=('LapNumber', 'count')).reset_index())

            results = race.results[['DriverNumber', 'Position']]

            stint_data = stint_data.merge(results, left_on='DriverNumber', right_on='DriverNumber', how='left')
            stint_data = stint_data.sort_values(by=['Position', 'Stint']).reset_index(drop=True)

            statuses = race.results['Status'].fillna('').astype(str)
            # dnf_mask = (statuses != 'Finished') & (statuses != 'Lapped') & (~statuses.str.startswith('+')) & (statuses != 'Disqualified')
            # dnf_drivers = race.results[dnf_mask]['Abbreviation'].tolist()

            # Add each driver's strategy to the strategy_df
            for _, driver in race.results.iterrows():
                driver_stints = stint_data[stint_data['DriverNumber'] == driver['DriverNumber']]
                stint_lengths = driver_stints['stint_length'].tolist()
                try:
                    stint_lengths[0] += 1
                except IndexError:
                    pass
                if sum(stint_lengths) != driver['Laps']:
                    try:
                        if sum(stint_lengths) < driver['Laps']:
                            stint_lengths[0] += driver['Laps'] - sum(stint_lengths)
                        else:
                            stint_lengths[0] -= sum(stint_lengths) - driver['Laps']
                        stint_lengths[0] = int(stint_lengths[0])
                    except IndexError:
                        pass
                stint_compounds_full = driver_stints['Compound'].tolist()
                stint_compounds = []
                for i in range(len(stint_compounds_full)):
                    if stint_compounds_full[i] == 'HYPERSOFT':
                        stint_compounds.append('HS')
                    elif stint_compounds_full[i] == 'ULTRASOFT':
                        stint_compounds.append('US')
                    elif stint_compounds_full[i] == 'SUPERSOFT':
                        stint_compounds.append('SS')
                    elif stint_compounds_full[i] == 'SOFT':
                        stint_compounds.append('S')
                    elif stint_compounds_full[i] == 'MEDIUM':
                        stint_compounds.append('M')
                    elif stint_compounds_full[i] == 'HARD':
                        stint_compounds.append('H')
                    elif stint_compounds_full[i] == 'INTERMEDIATE':
                        stint_compounds.append('I')
                    elif stint_compounds_full[i] == 'WET':
                        stint_compounds.append('W')

                total_stints = driver_stints['Stint'].nunique()

                if driver['ClassifiedPosition'] == 'W':
                    finish_position = 'DNS'
                elif driver['ClassifiedPosition'] == 'R':
                    finish_position = 'DNF'
                elif statuses[driver.name] == 'Disqualified':
                    finish_position = 'DSQ'
                else:
                    finish_position = int(driver['Position'])

                strategy_entry = {
                    'year': year,
                    'round': int(event['RoundNumber']),
                    'grand_prix': f"{event['EventName']}",
                    'finish_position': finish_position,
                    'number': driver['DriverNumber'],
                    'driver': driver['Abbreviation'],
                    'constructor': driver['TeamName'],
                    'stint_compounds': stint_compounds,
                    'stint_lengths': stint_lengths,
                    'total_stops': total_stints - 1 if total_stints > 0 else 0,
                    'total_laps': int(driver['Laps'])
                }

                strategy_df = pd.concat([strategy_df, pd.DataFrame([strategy_entry])], ignore_index=True)
                

        except Exception as e:
            print(f"Could not load race data for {event['EventName']} {year}: {e}")
            continue

core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 complet

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', None)
pd.set_option('display.max_columns', None)

strategy_df

,year,round,grand_prix,finish_position,number,driver,constructor,stint_compounds,stint_lengths,total_stops,total_laps
0,2018,1,Australian Grand Prix,1,5,VET,Ferrari,"[US, S]","[26, 32]",1,58
1,2018,1,Australian Grand Prix,2,44,HAM,Mercedes,"[US, S]","[19, 39]",1,58
2,2018,1,Australian Grand Prix,3,7,RAI,Ferrari,"[US, S]","[18, 40]",1,58
3,2018,1,Australian Grand Prix,4,3,RIC,Red Bull Racing,"[SS, S]","[26, 32]",1,58
4,2018,1,Australian Grand Prix,5,14,ALO,McLaren,"[US, S]","[26, 32]",1,58
5,2018,1,Australian Grand Prix,6,33,VER,Red Bull Racing,"[SS, S]","[21, 37]",1,58
6,2018,1,Australian Grand Prix,7,27,HUL,Renault,"[US, S]","[24, 34]",1,58
7,2018,1,Australian Grand Prix,8,77,BOT,Mercedes,"[US, SS]","[25, 33]",1,58
8,2018,1,Australian Grand Prix,9,2,VAN,McLaren,"[US, SS]","[25, 33]",1,58
9,2018,1,Australian Grand Prix,10,55,SAI,Renault,"[US, S]","[22, 36]",1,58


In [10]:
# Export to CSV
strategy_df.to_csv('f1_strategy_data.csv', index=False)

In [11]:
def get_yellow_flag_laps(race_status):
    laps = []
    for status in race_status.items():
        if '2' in str(status[1]) or '7' in str(status[1]):
            laps.append(int(status[0]))
    return laps

def get_red_flag_laps(race_status):
    laps = []
    for status in race_status.items():
        if '5' in str(status[1]):
            laps.append(int(status[0]))
    return laps 

def get_safety_car_laps(race_status):
    laps = []
    for status in race_status.items():
        if '4' in str(status[1]):
            laps.append(int(status[0]))
    return laps

def get_virtual_safety_car_laps(race_status):
    laps = []
    for status in race_status.items():
        if '6' in str(status[1]):
            laps.append(int(status[0]))
    return laps 

In [12]:
from available_compounds import get_available_compounds
gp_df = pd.DataFrame(columns=['year', 'round', 'grand_prix', 'available_compounds', 'yellow_flag_laps', 'safety_car_laps', 'virtual_safety_car_laps', 'red_flag_laps', 'total_laps'])

for year in range(2018, current_date.year + 1):
    schedule = fastf1.get_event_schedule(year)
    for _, event in schedule.iterrows():
        if event['EventFormat'] == 'testing':
            continue
        try:
            race = fastf1.get_session(year, event['EventName'], 'Race')
            race.load()

            race_date = race.date.to_pydatetime()
            if race_date > current_date:
                        break

            laps = race.laps
            available_compounds = get_available_compounds(year, event['RoundNumber'])
            race_status = laps.groupby('LapNumber')['TrackStatus'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)
            race_control_flags = race.race_control_messages[['Flag', 'Lap']]

            yellow_flag_laps = get_yellow_flag_laps(race_status)
            safety_car_laps = get_safety_car_laps(race_status)
            virtual_safety_car_laps = get_virtual_safety_car_laps(race_status)
            red_flag_laps = get_red_flag_laps(race_status)

            gp_df = pd.concat([gp_df, pd.DataFrame([{
                'year': year,
                'round': int(event['RoundNumber']),
                'grand_prix': f"{event['EventName']}",
                'available_compounds': available_compounds,
                'yellow_flag_laps': yellow_flag_laps,
                'safety_car_laps': safety_car_laps,
                'virtual_safety_car_laps': virtual_safety_car_laps,
                'red_flag_laps': red_flag_laps,
                'total_laps': int(laps['LapNumber'].max())
            }])], ignore_index=True)


        except Exception as e:
            print(f"Could not load race data for {event['EventName']} {year}: {e}")
            continue


            

core           INFO 	Loading data for Australian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '28'
req            INFO 	Using cached data for car_data
req            INFO 	No cached data found for position_data. Loading data...
_api           INFO 	Fetching position data...
core        WARNING 	Car position data is unavailable!
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 5 complet

In [13]:
gp_df
gp_df.to_csv('f1_grand_prix_data.csv', index=False)